In [1]:
import os
import pandas as pd
from keras.models import *
from keras.applications import *
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
def construct_generator(preprocess_function, img_size):
    batch_size = 32
    valid_test_gen = ImageDataGenerator(preprocessing_function = preprocess_function)
    test_data_generator = valid_test_gen.flow_from_directory('.\\imgs\\test',
                                                              target_size=img_size,
                                                              batch_size=batch_size,
                                                              shuffle=False,
                                                              class_mode=None)
    return test_data_generator

In [3]:
# submition file
if not os.path.exists('good_submission'):
    os.mkdir('good_submission')
file_names = os.listdir('good_submission')
pure_file_name_set = []
print(file_names)
for file_name in file_names:
    pure_file_name_set.append(file_name[:file_name.rfind('.')])
print(pure_file_name_set)

['InceptionV3_0.32014.csv', 'InceptionV3_L96_0.31699.csv', 'InceptionV3_L96_0.317.csv']
['InceptionV3_0.32014', 'InceptionV3_L96_0.31699', 'InceptionV3_L96_0.317']


In [4]:
for model_name in os.listdir('good_models'):
    pure_model_name = model_name[:model_name.rfind('.')]
    
    if pure_model_name not in pure_file_name_set:
        # Load model
        print(model_name)
        model = load_model('good_models\\'+model_name)
        if 'InceptionV3' in pure_model_name:
            test_data_generator = construct_generator(inceptionv3.preprocess_input, model.input_shape[1:-1])
        elif 'Xception' in pure_model_name:
            test_data_generator = construct_generator(xception.preprocess_input, model.input_shape[1:-1])
        elif 'ResNet50' in pure_model_name:
            test_data_generator = construct_generator(resnet50.preprocess_input, model.input_shape[1:-1])
        else:
            print('Unknown model!')
        # Predict
        df = pd.read_csv("submit_example.csv", index_col='img')
        preds = model.predict_generator(test_data_generator, steps=len(test_data_generator), verbose=1)
        for i, fname in enumerate(test_data_generator.filenames):
            index = fname[fname.rfind('\\')+1:]
            df.loc[index] = preds[i]

        df.to_csv('good_submission/'+pure_model_name+'.csv', float_format='%.3f')


ResNet50_0.35883.h5
(240,)
Found 79726 images belonging to 1 classes.
2492/2492 [==============================] - 728s 292ms/step
Xception_L20_0.28532.h
(299,)
Found 79726 images belonging to 1 classes.
2492/2492 [==============================] - 979s 393ms/step
Xception_L20_0.30757.h
(299,)
Found 79726 images belonging to 1 classes.
2492/2492 [==============================] - 980s 393ms/step
Xception_L96_0.32139.h
(299,)
Found 79726 images belonging to 1 classes.
2492/2492 [==============================] - 982s 394ms/step


In [34]:
a = (1,2,3,4)
print(a[1:-1])

(2, 3)
